<a href="https://colab.research.google.com/github/amien1410/amien-scrapers/blob/main/Events_Scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import re
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [11]:
def clean_string(string):
    # Remove unwanted characters like tabs and extra spaces
    cleaned_string = re.sub(r'[\t\n\r]+', '', string)  # Remove tabs, newlines, and carriage returns
    cleaned_string = re.sub(r'\s{2,}', ' ', cleaned_string)  # Replace multiple spaces with a single space
    cleaned_string = cleaned_string.strip()  # Strip leading/trailing spaces

    return cleaned_string

In [3]:
res = requests.get('https://www.visitberlin.de/en/event-calendar-berlin?page=0')
soup = BeautifulSoup(res.text, 'html.parser')
berlinEvents = soup.find_all('li', class_='l-list__item')
len(berlinEvents)

21

In [ ]:
# @title
events = []
base_url = "https://www.visitberlin.de"

for event in berlinEvents:
    # Extract event name, if available
    event_name = event.find('a', class_='teaser-search__mainlink')
    event_name = event_name.get('title', '').strip() if event_name else None

    # Extract picture link, if available
    picture_tag = event.find('img', class_='teaser-search__img fluid-img')
    picture_link = base_url + picture_tag.get('src') if picture_tag else None

    # Extract category, if available
    category_tag = event.find('div', class_='teaser-search__category')
    category = category_tag.get_text(strip=True) if category_tag else None

    # Extract address, if available
    address_tag = event.find('span', class_='teaser-search__print-address teaser-search__print-info')
    address = address_tag.get_text(strip=True) if address_tag else None

    # Extract website, if available
    website_tag = event.find('p', class_='teaser-search__print-link teaser-search__print-info')
    website = website_tag.get_text(strip=True) if website_tag else None

    # Extract time, if available
    time = ""
    time_tag = event.find('p', class_='teaser-search__time me')
    if time_tag:
        time = time_tag.select_one('span.me__content').get_text(strip=True)

    # Extract start and end dates, if available
    start_date, end_date = None, None
    date_tag = event.find('p', class_='teaser-search__date')
    if date_tag:
        times = date_tag.find_all('time')
        if len(times) > 1:
            start_date = times[0].get_text(strip=True)
            end_date = times[1].get_text(strip=True)
        elif len(times) == 1:
            start_date = times[0].get_text(strip=True)

    # Create the event info dictionary
    # print(time)
    event_info = {
        'event_name': event_name,
        'picture_link': picture_link,
        'category': category,
        'address': address,
        'city': 'Berlin',
        'country': 'Germany',
        'event_date': f"{start_date} - {end_date}" if start_date and end_date else start_date or None,
        'opening_hours': clean_string(time),
        'address': address,
        'ticket_price': "",
        'ticket_link': website,
        'source_website': base_url
    }

    # Add the event info to the events list
    events.append(event_info)

events

In [28]:
base_url = "https://www.visitberlin.de"
page = 0
events = []

while True:
    res = requests.get(f'{base_url}/en/event-calendar-berlin?page={page}')
    soup = BeautifulSoup(res.text, 'html.parser')
    berlinEvents = soup.find_all('li', class_='l-list__item')

    if not berlinEvents:  # If no events are found, break the loop
        break

    for event in berlinEvents:
        # Extract event name, if available
        event_name = event.find('a', class_='teaser-search__mainlink')
        event_name = event_name.get('title', '').strip() if event_name else None

        # Extract picture link, if available
        picture_tag = event.find('img', class_='teaser-search__img fluid-img')
        picture_link = base_url + picture_tag.get('src') if picture_tag else None

        # Extract category, if available
        category_tag = event.find('div', class_='teaser-search__category')
        category = category_tag.get_text(strip=True) if category_tag else None

        # Extract address, if available
        address_tag = event.find('span', class_='teaser-search__print-address teaser-search__print-info')
        address = address_tag.get_text(strip=True) if address_tag else None

        # Extract website, if available
        website_tag = event.find('p', class_='teaser-search__print-link teaser-search__print-info')
        website = website_tag.get_text(strip=True) if website_tag else None

        # Extract time, if available
        time = ""
        time_tag = event.find('p', class_='teaser-search__time me')
        if time_tag:
            time = time_tag.select_one('span.me__content').get_text(strip=True)

        # Extract start and end dates, if available
        start_date, end_date = None, None
        date_tag = event.find('p', class_='teaser-search__date')
        if date_tag:
            times = date_tag.find_all('time')
            if len(times) > 1:
                start_date = times[0].get_text(strip=True)
                end_date = times[1].get_text(strip=True)
            elif len(times) == 1:
                start_date = times[0].get_text(strip=True)

        # Create the event info dictionary
        event_info = {
            'event_name': event_name,
            'picture_link': picture_link,
            'category': category,
            'address': address,
            'city': 'Berlin',
            'country': 'Germany',
            'event_date': f"{start_date} - {end_date}" if start_date and end_date else start_date or None,
            'opening_hours': clean_string(time),
            'address': address,
            'ticket_price': "",
            'ticket_link': website,
            'source_website': base_url
        }

        # Add the event info to the events list
        events.append(event_info)
    print(page)
    page += 1  # Increment page for the next iteration

# Convert events list to Pandas DataFrame
df = pd.DataFrame(events)
df

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254


,event_name,picture_link,category,address,city,country,event_date,opening_hours,ticket_price,ticket_link,source_website
0,The Artist Cemetery,https://www.visitberlin.de/system/files/styles...,City Tours,None,Berlin,Germany,None,,,None,https://www.visitberlin.de
1,Let candles glow,https://www.visitberlin.de/system/files/styles...,Workshop,MACHmit! Museum für Kinder,Berlin,Germany,04/10/2024,10:00 am – 18:00 pm,,https://machmitmuseum.de,https://www.visitberlin.de
2,"Berlin, Du coole Sau! The Berlin-Revue",https://www.visitberlin.de/system/files/styles...,Show,Tipi am KanzleramtGroße Querallee10557 Berlin,Berlin,Germany,18/12/2024 - 12/01/2025,,,https://www.tipi-am-kanzleramt.de/de/programm/...,https://www.visitberlin.de
3,Special Workshops October MACHmit! Museum für ...,https://www.visitberlin.de/system/files/styles...,Workshop,MACHmit! Museum für Kinder,Berlin,Germany,04/10/2024,10:00 am – 18:00 pm,,https://machmitmuseum.de,https://www.visitberlin.de
4,AFTER NATURE,https://www.visitberlin.de/system/files/styles...,Exhibitions,"Humboldt Forum: Humboldt-Labor, 1. OG.Schlossp...",Berlin,Germany,04/10/2024,10:30 am – 18:30 pm,,https://www.humboldtforum.org/de/programm/daue...,https://www.visitberlin.de
...,...,...,...,...,...,...,...,...,...,...,...
5335,IKONO Berlin - Immersive Experience [100 Free ...,https://www.visitberlin.de/system/files/styles...,Exhibitions,None,Berlin,Germany,AB HERBST 2024 IN BERLIN!,,,https://ikono.global/de/berlin,https://www.visitberlin.de
5336,KlangKüken: Daytime or From Morning Allegro to...,https://www.visitberlin.de/system/files/styles...,Classical Music,None,Berlin,Germany,None,,,None,https://www.visitberlin.de
5337,Laraaji + Patrick Shiroishi,https://www.visitberlin.de/system/files/styles...,Konzerte,None,Berlin,Germany,None,,,None,https://www.visitberlin.de
5338,Organ tour in the Holy Cross Church,https://www.visitberlin.de/system/files/styles...,Guided Tour,None,Berlin,Germany,None,,,None,https://www.visitberlin.de


In [29]:
df = pd.DataFrame(events)
df.to_excel('berlin_events.xlsx', index=False)

In [23]:
hamburgUrl = "https://www.hamburg-travel.com/see-explore/events/events-calendar/js.api?filter[date]=05.10.2024&filter[district]=all&page=0&filter[distance]=50"
res = requests.get(hamburgUrl)
soup = BeautifulSoup(res.text, 'html.parser')
hamburgEvents = soup.find_all('article', class_='listTeaser-event')
len(hamburgEvents)

3

In [ ]:
events = []
base_url = "https://www.hamburg-travel.com/"
for event in hamburgEvents:
    # Extract event details
    event_name = event.find('h3').text if event.find('h3') else None
    event_type = event.find('ul', class_='listTeaser-event__text__profiling').text.strip() if event.find('ul', class_='listTeaser-event__text__profiling') else None
    date = event.find('span', class_='icon-calendar').next_sibling.strip() if event.find('span', class_='icon-calendar') else None
    time = event.find('span', class_='icon-clock').next_sibling.strip() if event.find('span', class_='icon-clock') else None
    location = event.find('span', class_='icon-located').next_sibling.strip() if event.find('span', class_='icon-located') else None
    picture_link = event.find('img')['src'] if event.find('img') else None
    imo_number = None
    departure_info = None

    # Check for IMO number and departure time
    text_sections = event.find_all('p')
    for p in text_sections:
        if 'IMO number' in p.text:
            imo_number = p.text.split('IMO number: ')[1].split()[0] if 'IMO number' in p.text else None
        if 'Departure' in p.text:
            departure_info = event.find('ul').find('li').text.strip()

    event_info = {
        'event_name': event_name,
        'picture_link': picture_link,
        'event_type': event_type,
        'date': date,
        'time': time,
        'adress': location,
        'city': 'Hamburg',
        'country': 'Germany',
        'ticket_price': "",
        'ticket_link': "",
        'source_website': base_url,
        'note': departure_info
    }
    events.append(event_info)

events

In [31]:
# Base URL with placeholders for dynamic page number
base_url = "https://www.hamburg-travel.com/"
hamburgUrl = "https://www.hamburg-travel.com/see-explore/events/events-calendar/js.api?filter[district]=all&pageDate=05.10.2024&page={}&filter[distance]=50"

events = []
page = 0

while True:
    # Fetch page content
    url = hamburgUrl.format(page)
    print(url)
    res = requests.get(url)
    soup = BeautifulSoup(res.text, 'html.parser')
    hamburgEvents = soup.find_all('article', class_='listTeaser-event')

    if not hamburgEvents:  # Break loop if no more events are found
        break

    for event in hamburgEvents:
        # Extract event details
        event_name = event.find('h3').text if event.find('h3') else None
        event_type = event.find('ul', class_='listTeaser-event__text__profiling').text.strip() if event.find('ul', class_='listTeaser-event__text__profiling') else None
        date = event.find('span', class_='icon-calendar').next_sibling.strip() if event.find('span', class_='icon-calendar') else None
        time = event.find('span', class_='icon-clock').next_sibling.strip() if event.find('span', class_='icon-clock') else None
        location = event.find('span', class_='icon-located').next_sibling.strip() if event.find('span', class_='icon-located') else None
        picture_link = event.find('img')['src'] if event.find('img') else None
        imo_number = None
        departure_info = None

        # Check for IMO number and departure time
        text_sections = event.find_all('p')
        for p in text_sections:
            if 'IMO number' in p.text:
                imo_number = p.text.split('IMO number: ')[1].split()[0] if 'IMO number' in p.text else None
            if 'Departure' in p.text:
                departure_info = event.find('ul').find('li').text.strip()

        # Create event info dictionary
        event_info = {
            'event_name': event_name,
            'picture_link': picture_link,
            'event_type': event_type,
            'date': date,
            'time': time,
            'adress': location,
            'city': 'Hamburg',
            'country': 'Germany',
            'ticket_price': "",
            'ticket_link': "",
            'source_website': base_url,
            'note': departure_info
        }

        events.append(event_info)

    # Move to the next page
    print(page)
    page += 1

# All events have been scraped
print(f"Total events scraped: {len(events)}")

https://www.hamburg-travel.com/see-explore/events/events-calendar/js.api?filter[district]=all&pageDate=05.10.2024&page=0&filter[distance]=50
0
https://www.hamburg-travel.com/see-explore/events/events-calendar/js.api?filter[district]=all&pageDate=05.10.2024&page=1&filter[distance]=50
1
https://www.hamburg-travel.com/see-explore/events/events-calendar/js.api?filter[district]=all&pageDate=05.10.2024&page=2&filter[distance]=50
2
https://www.hamburg-travel.com/see-explore/events/events-calendar/js.api?filter[district]=all&pageDate=05.10.2024&page=3&filter[distance]=50
3
https://www.hamburg-travel.com/see-explore/events/events-calendar/js.api?filter[district]=all&pageDate=05.10.2024&page=4&filter[distance]=50
4
https://www.hamburg-travel.com/see-explore/events/events-calendar/js.api?filter[district]=all&pageDate=05.10.2024&page=5&filter[distance]=50
5
https://www.hamburg-travel.com/see-explore/events/events-calendar/js.api?filter[district]=all&pageDate=05.10.2024&page=6&filter[distance]=50
6

In [33]:
# df = pd.DataFrame(events)
# df.to_excel('hamburg_events.xlsx', index=False)
df

,event_name,picture_link,event_type,date,time,adress,city,country,ticket_price,ticket_link,source_website,note
0,Flohdom Trabrennbahn Bahrenfeld,https://www.hamburg-tourism.de/images/19107UOs...,Flea Markets,05.10.2024,07:00,Trabrennbahn Bahrenfeld,Hamburg,Germany,,,https://www.hamburg-travel.com/,None
1,Wochenmarkt in der Wedeler Landstraße,https://www.hamburg-tourism.de/images/-sRYgxuu...,Farmer's Markets,05.10.2024,07:00,Wedeler Landstraße,Hamburg,Germany,,,https://www.hamburg-travel.com/,None
2,Lauenburger Wochenmarkt,https://www.hamburg-tourism.de/images/WmlhkI-u...,Farmer's Markets,05.10.2024,07:00,Marktplatz Lauenburg,Hamburg,Germany,,,https://www.hamburg-travel.com/,None
3,VitalMarkt,https://www.hamburg-tourism.de/images/CblUZMun...,Farmer's Markets,05.10.2024,07:00,Rathauskreuzung Uelzen,Hamburg,Germany,,,https://www.hamburg-travel.com/,None
4,Wochenmarkt auf dem Stübenplatz,https://www.hamburg-tourism.de/images/OriirfbT...,Farmer's Markets,05.10.2024,07:00,Stübenplatz,Hamburg,Germany,,,https://www.hamburg-travel.com/,None
...,...,...,...,...,...,...,...,...,...,...,...,...
108,Disneys HERCULES,https://www.hamburg-tourism.de/images/ipqFNKL2...,Musicals,05.10.2024,19:30,Stage Theater Neue Flora,Hamburg,Germany,,,https://www.hamburg-travel.com/,None
109,Disneys DER KÖNIG DER LÖWEN,https://www.hamburg-tourism.de/images/9MOmxX_t...,Musicals,05.10.2024,20:00,Stage Theater im Hafen,Hamburg,Germany,,,https://www.hamburg-travel.com/,None
110,Heiße Ecke,https://www.hamburg-tourism.de/images/ijz49Qzp...,Musicals,05.10.2024,20:00,Schmidts TIVOLI,Hamburg,Germany,,,https://www.hamburg-travel.com/,None
111,PubCrawl Hamburg,https://www.hamburg-tourism.de/images/S98VgXVY...,Nightlife,05.10.2024,21:00,Treffpunkt: Davidwache,Hamburg,Germany,,,https://www.hamburg-travel.com/,None


In [ ]:
muenchenUrl = "https://www.muenchen.de/en/veranstaltungen/event?page=0"
res = requests.get(muenchenUrl)
soup = BeautifulSoup(res.text, 'html.parser')
muenchenEvents = soup.find_all('li', class_='m-listing__list-item')
len(muenchenEvents)

30

In [ ]:
events = []
for event in muenchenEvents:
  # Extract event name
  event_name = event.find('h3', class_='m-event-list-item__headline').get_text(strip=True)

  # Extract start and end dates
  start_date = event.find('time', class_='m-date-range__item', itemprop='startDate')['datetime']
  end_date = event.find('time', class_='m-date-range__item', itemprop='endDate')['datetime']

  # Extract event day and time
  date_time_tag = event.find('time', datetime=True)
  event_datetime = date_time_tag['datetime'] if date_time_tag else None

  # Extract location
  location_tag = event.find('p', class_='m-event-list-item__detail', itemprop='location')
  location = location_tag.get_text(strip=True) if location_tag else None

  # Extract ticket link
  ticket_link_tag = event.find('a', class_='m-button', href=True)
  ticket_link = ticket_link_tag['href'] if ticket_link_tag else None

  event_info = {
      'event_name': event_name,
      'start_date': start_date,
      'end_date': end_date,
      'event_datetime': event_datetime,
      'location': location,
      'ticket_link': ticket_link
  }
  events.append(event_info)

events

[{'event_name': 'Tanzkurs Zwiefache mit Live-Musik',
  'start_date': '2024-10-02T12:00:00Z',
  'end_date': '2024-10-16T12:00:00Z',
  'event_datetime': '2024-10-02T12:00:00Z',
  'location': 'Kulturzentrum LUISE',
  'ticket_link': 'https://www.muenchenticket.de/tickets/performances/ei8udu5xbyt7/Tanzkurs-Zwiefache-mit-Live-Musik-3-Tages-Ticket?campaign=muenchen'},
 {'event_name': 'Tintenfische, Teufelsfinger und Tentakel - die faszinierende Welt der Kopffüßer',
  'start_date': '2024-08-13T12:00:00Z',
  'end_date': '2024-12-30T12:00:00Z',
  'event_datetime': '2024-08-13T12:00:00Z',
  'location': 'Paläontologisches Museum',
  'ticket_link': None},
 {'event_name': 'Skelette – Choreografen der Bewegung',
  'start_date': '2024-07-17T12:00:00Z',
  'end_date': '2024-10-15T12:00:00Z',
  'event_datetime': '2024-07-17T12:00:00Z',
  'location': 'Museum Mensch und Natur',
  'ticket_link': None},
 {'event_name': 'Sonderausstellung Erntedank: Von Erbsenzählern und Bohnenstangen – die Vielfalt der regio